# IMPORT LIBRARIES

In [1]:
import pandas as pd

from datetime import datetime

import MetaTrader5 as mt5

import pytz

# CONFIGURATION

In [2]:
gc_o_TIME_ZONE = pytz.timezone("Etc/UTC")
gc_dt_FROM = datetime(2021, 9, 1, tzinfo=gc_o_TIME_ZONE)
gc_dt_TO = datetime(2022, 3, 10, tzinfo=gc_o_TIME_ZONE)

In [3]:
dfCrpytocurrencies = pd.read_csv('Static Data\cryptocurrencies.csv')

# FETCH FROM METATRADER 5 

In [4]:
def aFetchFromMT5(sSymbol,dtFrom, dtTo, oFreq):
    if not mt5.initialize():
        print("initialize() failed, error code =", mt5.last_error())
        sys.exit()

    aSymbolInfo = mt5.symbol_info(sSymbol)
    if not aSymbolInfo:
        print("symbol_info() failed, error code =", mt5.last_error())
        sys.exit()

    aOhlcSample = mt5.copy_rates_range(
        sSymbol,
        oFreq,
        dtFrom, 
        dtTo
    )

    if len(aOhlcSample) == 0:
        print("copy_rates_range() failed, error code =", mt5.last_error())
        sys.exit()

    mt5.shutdown()
    return aOhlcSample

In [5]:
aDatesToFetch = list(pd.date_range(start=gc_dt_FROM, end=gc_dt_TO)) #created since MT5 library fails due to time out.
aDatesSampled = aDatesToFetch[::50]
aDatesSampled.append(aDatesToFetch[-1])
aDatesSampled = list(set(aDatesSampled))
aDatesSampled.sort()

dfOhlcSource = pd.DataFrame()
for sSymbol in dfCrpytocurrencies['Symbol'].values:
    for i in range(0, len(aDatesSampled) - 1):
        dtFrom = aDatesSampled[i]
        dtTo = aDatesSampled[i+1]

        aOhlcSample = aFetchFromMT5(sSymbol,dtFrom, dtTo, oFreq = mt5.TIMEFRAME_M30)

        dfOhlcSample = pd.DataFrame(aOhlcSample)
        dfOhlcSample['symbol'] = sSymbol

        dfOhlcSample['timestamp'] = pd.to_datetime(dfOhlcSample['time'], unit= "s")
        dfOhlcSample.set_index('timestamp', inplace=True)
        dfOhlcSample.drop(["time"], axis = 1 , inplace = True)

        dfOhlcSource = dfOhlcSource.append(dfOhlcSample)

dfOhlcSource.drop_duplicates(inplace = True)

# COMPILE MARKET DATA

## Add Seasonal Features

In [6]:
dfOhlcSource["weekday"] = dfOhlcSource.index.weekday
dfOhlcSource["hour"] = dfOhlcSource.index.hour
dfOhlcSource["minute"] = dfOhlcSource.index.minute

## Add [Return] Feature

In [7]:
dfOhlcSource["return"] = (dfOhlcSource["close"] - dfOhlcSource["open"])/dfOhlcSource["open"]

## Add Candle Features

In [8]:
dfOhlcSource["upper_shadow"] =( dfOhlcSource["high"] - dfOhlcSource[['close', 'open']].max(axis=1))/ dfOhlcSource[['close', 'open']].max(axis=1)
dfOhlcSource["lower_shadow"] = (dfOhlcSource[['close', 'open']].min(axis=1) - dfOhlcSource["low"])/dfOhlcSource["low"]

## Transform Symbols to Columns

In [9]:
dfOhlc = pd.DataFrame()

i = 1
for sSymbol in dfCrpytocurrencies['Symbol'].values:
    dfSymbolValues = dfOhlcSource[dfOhlcSource['symbol'] == sSymbol]

    if i == 1:
        sHow = "right"
    else:
        sHow = "inner"
    
    dfSymbolValues = dfSymbolValues.drop('symbol', axis = 1)
    
    dfOhlc = dfOhlc.join(dfSymbolValues,how = sHow, rsuffix=sSymbol)
    
    i = i + 1
    
aColumnsOhlc = list()
for sSymbol in dfCrpytocurrencies['Symbol'].values:
    for sColumn in dfOhlcSource.columns:
        if sColumn != 'symbol':
            sNewColumn = sSymbol + ":" + sColumn
            aColumnsOhlc.append(sNewColumn)
    
dfOhlc.columns = aColumnsOhlc
dfOhlc.to_csv('Static Data\dfOhlc.csv')

In [10]:
dfOhlc

BTCUSD:open  BTCUSD:high  BTCUSD:low  BTCUSD:close  \
timestamp                                                                 
2021-09-01 00:00:00     46896.00     47101.26    46835.53      46987.03   
2021-09-01 00:30:00     46981.03     47200.50    46912.22      46912.22   
2021-09-01 01:00:00     46914.65     47013.10    46760.81      46857.08   
2021-09-01 01:30:00     46857.08     47114.02    46755.01      47092.37   
2021-09-01 02:00:00     47092.37     47257.26    46997.88      47201.51   
...                          ...          ...         ...           ...   
2022-03-09 22:00:00     42217.74     42340.31    42042.22      42103.93   
2022-03-09 22:30:00     42103.93     42135.43    41690.22      41841.23   
2022-03-09 23:00:00     41841.23     41952.33    41680.38      41807.19   
2022-03-09 23:30:00     41806.19     41957.01    41707.29      41901.55   
2022-03-10 00:00:00     41927.05     42009.01    41854.11      41982.61   

                     BTCUSD:tick_volume  BTCUSD:spread  BTCUSD:real_volume  \
timestamp                                                                    
2021-09-01 00:00:00                2249            911                   0   
2021-09-01 00:30:00                3140           1179                   0   
2021-09-01 01:00:00                4257           1050                   0   
2021-09-01 01:30:00                3926           1050                   0   
2021-09-01 02:00:00                3587           1050                   0   
...                                 ...            ...                 ...   
2022-03-09 22:00:00                1624           1000                   0   
2022-03-09 22:30:00                1913           1000                   0   
2022-03-09 23:00:00                1428           1000                   0   
2022-03-09 23:30:00                1579           1000                   0   
2022-03-10 00:00:00                1061           1000                   0   

                     BTCUSD:weekday  BTCUSD:hour  BTCUSD:minute  ...  \
timestamp                                                        ...   
2021-09-01 00:00:00               2            0              0  ...   
2021-09-01 00:30:00               2            0             30  ...   
2021-09-01 01:00:00               2            1              0  ...   
2021-09-01 01:30:00               2            1             30  ...   
2021-09-01 02:00:00               2            2              0  ...   
...                             ...          ...            ...  ...   
2022-03-09 22:00:00               2           22              0  ...   
2022-03-09 22:30:00               2           22             30  ...   
2022-03-09 23:00:00               2           23              0  ...   
2022-03-09 23:30:00               2           23             30  ...   
2022-03-10 00:00:00               3            0              0  ...   

                     RPLUSD:close  RPLUSD:tick_volume  RPLUSD:spread  \
timestamp                                                              
2021-09-01 00:00:00        1.1779                 597             30   
2021-09-01 00:30:00        1.1773                1356             36   
2021-09-01 01:00:00        1.1568                1434             31   
2021-09-01 01:30:00        1.1714                1365             30   
2021-09-01 02:00:00        1.1763                1422             30   
...                           ...                 ...            ...   
2022-03-09 22:00:00        0.7617                 590             25   
2022-03-09 22:30:00        0.7600                1000             26   
2022-03-09 23:00:00        0.7605                 422             25   
2022-03-09 23:30:00        0.7616                 406             27   
2022-03-10 00:00:00        0.7627                 194             28   

                     RPLUSD:real_volume  RPLUSD:weekday  RPLUSD:hour  \
timestamp                                                              
2021-09-01 0